In [2]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import numpy as np

# Competitor's Web Scraping

## H&M - Men - Jeans - showroom (main) page

In [3]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0'}
# encontro esse user-agent da linha acima em http://developers.whatismybrowser.com/

In [5]:
page = requests.get(url, headers=headers)

In [6]:
soup = BeautifulSoup(page.text, 'html.parser')

In [7]:
products = soup.find('ul', class_='products-listing small')  # not find_all because I want only the first 

In [8]:
product_list = products.find_all('article', class_='hm-product-item')

In [9]:
# fazer um laco for que percorre a lista aplicando o metodo get sobre o "data-articlecode"
# escrevendo o for numa linha unica de codigo que vai retornar uma lista

# product_id
product_id = [p.get('data-articlecode') for p in product_list]

# product_category
product_category = [p.get('data-category') for p in product_list]

In [10]:
# product_name
product_list = products.find_all('a', class_='link')
product_name = [p.get_text() for p in product_list]

In [11]:
# price
product_list = products.find_all('span', class_='price regular')
product_price = [p.get_text() for p in product_list]

In [12]:
data = pd.DataFrame([product_id, product_category, product_name, product_price]).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']

In [13]:
data.head()

,product_id,product_category,product_name,product_price
0,1024256001,men_jeans_slim,Slim Jeans,$ 19.99
1,1008549006,men_jeans_regular,Regular Jeans,$ 19.99
2,1024256002,men_jeans_slim,Slim Jeans,$ 19.99
3,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99
4,0690449051,men_jeans_ripped,Skinny Jeans,$ 39.99


In [36]:
data['style_id'] = data['product_id'].apply(lambda x: x[:-3])
data['color_id'] = data['product_id'].apply(lambda x: x[-3:])
data.head()

,product_id,product_category,product_name,product_price,style_id,color_id
0,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,1024256,001
1,1008549006,men_jeans_regular,Regular Jeans,$ 19.99,1008549,006
2,1024256002,men_jeans_slim,Slim Jeans,$ 19.99,1024256,002
3,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,043
4,0690449051,men_jeans_ripped,Skinny Jeans,$ 39.99,0690449,051


## H&M - Men - Jeans - product individual pages

In [15]:
# API Requests

# example of a product page
url_3 = 'https://www2.hm.com/en_us/productpage.0985159001.html'

page = requests.get(url_3, headers=headers)

# Beautiful Soup object
soup = BeautifulSoup(page.text, 'html.parser')

# ================ Product Color ==================

gen_color_list = soup.find_all('a', class_='filter-option miniature')

# product colors
product_colors = [c.get('data-color') for c in gen_color_list]

# product id
product_id = [c.get('data-articlecode') for c in gen_color_list]

# dataframe color
df_color = pd.DataFrame(list(zip(product_id, product_colors)))
df_color.columns = ['product_id', 'product_colors']
df_color.head()

# generate style id + color id
df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])
df_color.head()

# ================ Product Composition ==================

gen_composition_list = soup.find_all('div', class_='details-attributes-list-item')
gen_composition = [list(filter(None, d.get_text().split('\n'))) for d in gen_composition_list]
gen_composition

del gen_composition[5]

df_composition = pd.DataFrame(gen_composition).T

# rename column names
df_composition.columns = df_composition.iloc[0]

# delete first row
df_composition = df_composition.iloc[1:]

df_composition = df_composition.loc[:, ['Fit', 'Composition', 'Art. No.']]

# fill NA with info from the row above
df_composition = df_composition.fillna(method='ffill')

# extract style id and article id from product_id
# generate style id + composition id
df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
df_composition['article_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])
df_composition.head()

# Merge color and composition dataframes
data_sku = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition']], how='left', on='style_id')
data_sku.head()


,product_id,product_colors,style_id,color_id,Fit,Composition
0,0985159002,Denim blue,0985159,002,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
1,0985159002,Denim blue,0985159,002,Skinny fit,Pocket lining: Cotton 100%
2,0985159003,Dark gray,0985159,003,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
3,0985159003,Dark gray,0985159,003,Skinny fit,Pocket lining: Cotton 100%
4,0985159004,Light denim blue,0985159,004,Skinny fit,"Shell: Cotton 99%, Spandex 1%"


### H&M - Men - Jeans - multiple pages¶

In [16]:
# product_url = ['https://www2.hm.com/en_us/productpage.' + str(p) + '.html' for p in product_id]

In [34]:
# empty dataframe
df_details = pd.DataFrame()

# empty list to collect names of unique columns for all products composition
aux = []

# list of columns I selected from set(aux) after all iteractions
cols = ['Art. No.', 'Collection', 'Composition', 'Description', 'Fit', 'Imported'] 

# pattern dataframe with the columns I want to collect from each product if exist
df_pattern = pd.DataFrame(columns=cols)

for i in range(len(data)):
   
    # API Requests
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
    page = requests.get(url, headers=headers)

    # Beautiful Soup object
    soup = BeautifulSoup(page.text, 'html.parser')

    # ================ Product Color ==================

    gen_color_list = soup.find_all('a', class_='filter-option miniature')

    # product colors
    product_colors = [c.get('data-color') for c in gen_color_list]

    # product id
    product_id = [c.get('data-articlecode') for c in gen_color_list]

    # dataframe color
    df_color = pd.DataFrame(list(zip(product_id, product_colors)))
    df_color.columns = ['product_id', 'product_colors']

    # generate style id + color id
    df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
    df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])
    

    # ================ Product Composition ==================

    gen_composition_list = soup.find_all('div', class_='details-attributes-list-item')
    gen_composition = [list(filter(None, d.get_text().split('\n'))) for d in gen_composition_list]
    
    df_composition = pd.DataFrame(gen_composition).T

    # rename column names
    df_composition.columns = df_composition.iloc[0]

    # delete first row
    df_composition = df_composition.iloc[1:]
    
    # guarantee the same selected columns for all products
    aux = aux + df_composition.columns.tolist()
    df_composition = pd.concat([df_pattern, df_composition], axis=0)

    #df_composition = df_composition.loc[:, ['Fit', 'Composition', 'Art. No.']]

    # fill NA with info from the row above
    df_composition = df_composition.fillna(method='ffill')

    # extract style id and article id from product_id
    # generate style id + composition id
    df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
    df_composition['article_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])
    
    

    # Merge color and composition dataframes
    data_sku = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition', 'Collection', 'Description', 'Imported']], how='left', on='style_id')

    # all product details
    df_details = pd.concat([df_details, data_sku], axis=0)
    

In [26]:
set(aux)

{'Art. No.',
 'Care instructions',
 'Collection',
 'Composition',
 'Concept',
 'Description',
 'Fit',
 'Imported',
 'Material',
 'More sustainable materials',
 'Nice to know',
 'messages.clothingStyle',
 'messages.garmentLength',
 'messages.waistRise'}

In [35]:
df_details.head()

,product_id,product_colors,style_id,color_id,Fit,Composition,Collection,Description,Imported
0,1024256002,Light denim blue,1024256,002,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,Black,Yes
1,1024256002,Light denim blue,1024256,002,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,Solid-color,Yes
2,1024256002,Light denim blue,1024256,002,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,Solid-color,Yes
3,1024256002,Light denim blue,1024256,002,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,Solid-color,Yes
4,1024256002,Light denim blue,1024256,002,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,Solid-color,Yes


## Merge data from showroom and product individual pages

In [40]:
data_raw = pd.merge(data, df_details[['style_id', 'product_colors', 'Fit', 'Composition', 
                                      'Collection', 'Description', 'Imported']], how='left', on='style_id')

In [42]:
data_raw.head()

,product_id,product_category,product_name,product_price,style_id,color_id,product_colors,Fit,Composition,Collection,Description,Imported
0,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,1024256,001,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,Black,Yes
1,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,1024256,001,Light denim blue,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,Solid-color,Yes
2,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,1024256,001,Light denim blue,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,Solid-color,Yes
3,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,1024256,001,Light denim blue,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,Solid-color,Yes
4,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,1024256,001,Light denim blue,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,Solid-color,Yes
